In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm

from msna.our_method.msna_pipeline import MSNA_pipeline
from msna.msna_common import get_files, read_msna
# sys.path.append("../other_methods")
# from wavelet import WaveletPipeline

/Users/trippdow/Projects/HealthIntelligence/msna/msna/our_method/msna_pipeline.py:49: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.cnn.load_state_dict(torch.load(f"{ou

In [16]:
 
files = get_files("../../../Archival/MSNAS/MSNA*/MSNA*burstcomments*.txt")
pipeline = MSNA_pipeline(sampling_rate=250, verbose=True)
# pipeline = WaveletPipeline(scale=10, verbose=True)
pipeline.load("../our_method/pretrained/")

In [ ]:

# np.random.shuffle(files)
for filename in tqdm(files[9:10]):
    df = read_msna(filename)

    if df is not None:
        
        # Extract indices of the actual BURST events
        actual_burst_indices = df[df['BURST'] != 0].index
        
        # predicted_burst_indices, probabilities = pipeline.predict(filename)
        predicted_burst_indices = pipeline.predict(filename)

        # probabilities = (probabilities - np.min(probabilities)) / (np.max(probabilities) - np.min(probabilities))
        
        plot_start = 3000 #np.random.randint(1000, 2000)
        plot_end = 5000 #plot_start + np.random.randint(20000, 30000)

        actual_burst_indices = actual_burst_indices[(actual_burst_indices<plot_end) & (actual_burst_indices>plot_start)]
        
        valid_indices = (predicted_burst_indices < plot_end) & (predicted_burst_indices > plot_start)
        predicted_burst_indices = predicted_burst_indices[valid_indices]
        # probabilities = probabilities[valid_indices]
        # new_probabilities = np.zeros(len(df))
        # new_probabilities[predicted_burst_indices] = probabilities
        # new_probabilities = new_probabilities[plot_start:plot_end]
        signal = df['Integrated MSNA'][plot_start:plot_end]
        
        # Plot the results
        plt.figure(figsize=(14, 8))
        plt.plot(signal, label='Filtered Signal')
        plt.plot(actual_burst_indices, 
                 df['Integrated MSNA'][actual_burst_indices], 
                 "o", label='Actual BURST Events', color='blue')
        plt.plot(predicted_burst_indices, 
                 df['Integrated MSNA'][predicted_burst_indices], 
                 "x", label='Detected Peaks', color='red')
        # plt.plot(range(plot_start, plot_end), 
        #          new_probabilities*(max(signal)-min(signal)) + min(signal), 
        #         label='Probabilities', color='red')
        
        plt.title('Detected Peaks vs Actual BURST Events')
        plt.xlabel('Sample Index')
        plt.ylabel('Filtered Signal')
        plt.legend()
        plt.show()


  0%|                                                     | 0/1 [00:00<?, ?it/s]